In [29]:
import requests
from pydantic import BaseModel
from typing import Optional


class SimilarAPIsRequest(BaseModel):
    prompt: str
    user_id: str
    k: Optional[int] = 10

class DocumentRequest(BaseModel):
    id: str


HOST = "192.168.49.2"
PORT = 32492

# HOST = "127.0.0.1"
# PORT = 8080

In [30]:
def test_get_similar_apis():
    endpoint = f"http://{HOST}:{PORT}/get_similar_apis"

    payload = SimilarAPIsRequest(prompt="How to blur people face?", user_id="test_user", k=3)

    return requests.post(endpoint, json=payload.model_dump())
wow = test_get_similar_apis()

In [35]:
wow = test_get_similar_apis()
wow

<Response [200]>

In [28]:
wow.json()

{'detail': [{'type': 'missing',
   'loc': ['body', 'openai_key'],
   'msg': 'Field required',
   'input': {'prompt': 'How to blur people face?',
    'user_id': 'test_user',
    'k': 3},
   'url': 'https://errors.pydantic.dev/2.6/v/missing'}]}

In [61]:
def test_get_status():
    endpoint = f"http://{HOST}:{PORT}/status"


    return requests.get(endpoint)
test_get_status()

<Response [200]>

In [62]:
wow = test_get_similar_apis()
wow.json()

{'documents': [['{"API_Title": "Image Blur API.", "API_Description": " This service blurs the image in the given areas. The areas are given as a list of [x1, y1, x2, y2] coordinates. ", "API_Version": "1.0.0", "OpenAPI_Version": "3.1.0", "Route": "/compute", "Method": "POST", "Endpoint_Description": "Compute task", "Parameters": {}, "Tags": ["Tasks"], "Responses": {"200": {"description": "Successful Response", "content": {"application/json": {"schema": {}}}}, "422": {"description": "Validation Error", "content": {"application/json": {"schema": {"$ref": "#/components/schemas/HTTPValidationError"}}}}}}',
   '{"API_Title": "Image Blur API.", "API_Description": " This service blurs the image in the given areas. The areas are given as a list of [x1, y1, x2, y2] coordinates. ", "API_Version": "1.0.0", "OpenAPI_Version": "3.1.0", "Route": "/tasks/{task_id}/status", "Method": "GET", "Endpoint_Description": "Get task status", "Parameters": {"task_id": {"name": "task_id", "in": "path", "required

**Connect to CHROMA in the cluster**

In [2]:
import chromadb


host = '192.168.49.2'
port = 31791
bearer_token = 'Hnq6GBxJlHfJEGoxrazxXdoG3aQ0gncP'
chroma_client = chromadb.HttpClient(
    host=host,
    port=port,
    headers={"Authorization": f"Bearer {bearer_token}"}
)



collection = chroma_client.get_collection("api_endpoints")

In [48]:
import joblib
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")


In [49]:
API_KEY = os.getenv("OPENAI_API_KEY")

In [50]:

request = "Je fais du sudoku et je dois détecter des nombres"



client = OpenAI(api_key=API_KEY)


def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

REQUEST = False
if REQUEST:
    embedding = get_embedding(request)
else:    
    joblib.dump(embedding, "../data/request_embedding.joblib")
    embedding = joblib.load("../data/request_embedding.joblib")

results = collection.query(query_embeddings = embedding)

In [51]:
results['ids'][0]

['348c4971-4f9d-4352-8ceb-e7f15ee5dd0a',
 '2ce73022-e17a-41c3-8ad6-59fe6833dc83',
 '74d3d802-5752-43eb-a440-2e918ba87da1',
 'c83b0b3a-8c9e-45e3-8201-60c1784aac13',
 '5b0c7d03-3d70-4add-9b37-3b2954d67c0f',
 '1295cb27-da33-4a8c-b919-7e8768776c10',
 '87d1ecc9-0e46-4557-8add-b15c05cfae94',
 'cb8f9d2f-01e4-4e0e-a4df-afb476c2248e',
 '76931b55-efe3-4ecd-a3d8-7e162fd7b8d9',
 '4aa2bc9e-050a-4832-b66f-0d7d0ef03d04']

In [52]:
collection.get(results['ids'][0][0])

{'ids': ['348c4971-4f9d-4352-8ceb-e7f15ee5dd0a'],
 'embeddings': None,
 'metadatas': [{'endpoint': '/compute',
   'http_method': 'POST',
   'openapi': '3.1.0',
   'title': 'Digit Recognition API.',
   'version': '1.0.0'}],
 'documents': ['{"API_Title": "Digit Recognition API.", "API_Description": " Recognizes a digit in an image using mnist trained model. ", "API_Version": "1.0.0", "OpenAPI_Version": "3.1.0", "Route": "/compute", "Method": "POST", "Endpoint_Description": "Compute task", "Parameters": {}, "Tags": ["Tasks"], "Responses": {"200": {"description": "Successful Response", "content": {"application/json": {"schema": {}}}}, "422": {"description": "Validation Error", "content": {"application/json": {"schema": {"$ref": "#/components/schemas/HTTPValidationError"}}}}}}'],
 'data': None,
 'uris': None}

In [55]:
collection.query(query_embeddings = embedding, n_results=3)

{'ids': [['348c4971-4f9d-4352-8ceb-e7f15ee5dd0a',
   '2ce73022-e17a-41c3-8ad6-59fe6833dc83',
   '74d3d802-5752-43eb-a440-2e918ba87da1']],
 'distances': [[1.5292056029921062, 1.5532544915307023, 1.5935734512086184]],
 'embeddings': None,
 'metadatas': [[{'endpoint': '/compute',
    'http_method': 'POST',
    'openapi': '3.1.0',
    'title': 'Digit Recognition API.',
    'version': '1.0.0'},
   {'endpoint': '/tasks/{task_id}/status',
    'http_method': 'GET',
    'openapi': '3.1.0',
    'title': 'Digit Recognition API.',
    'version': '1.0.0'},
   {'endpoint': '/compute',
    'http_method': 'POST',
    'openapi': '3.1.0',
    'title': 'NSFW Image Detection API.',
    'version': '0.2.1'}]],
 'documents': [['{"API_Title": "Digit Recognition API.", "API_Description": " Recognizes a digit in an image using mnist trained model. ", "API_Version": "1.0.0", "OpenAPI_Version": "3.1.0", "Route": "/compute", "Method": "POST", "Endpoint_Description": "Compute task", "Parameters": {}, "Tags": ["Task